# Submission for Part-2 
## Adding Location Data to the DataFrame

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim 

### requests library lets us get the html part of a website, .text retrives all its text content

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### BeautifulSoup function helps working with website scrapping easier, it has methods like pretiffy() that makes the content clearer

In [3]:
soup=BeautifulSoup(source, 'lxml')

## The contents needed are present in the soup under table

In [4]:
table = soup.find('table').text

### The values of the table are all separated by '\n\n' in the string 'table'

In [5]:
values = table.split('\n\n')

### Remove the headings of the table from the list

In [6]:
col=values[1:4]
col

['Postal Code', 'Borough', 'Neighborhood']

In [7]:
values=values[4:]

## The values are to be split into various columns.

In [8]:
length = len(values)
length

540

### There are total 540 values meaning that there are 180 values for each column.

### a,b and c are the indices of values of each column respectively.

In [9]:
col1=[]
col2=[]
col3=[]
a=0
b=1
c=2
while a<180:
    col1.append(values[a])
    col2.append(values[b])
    col3.append(values[c])
    a=a+3
    b=b+3
    c=c+3

In [10]:
col1[:5]

['\nM1A', '\nM2A', '\nM3A', '\nM4A', '\nM5A']

In [11]:
col2[:5]

['Not assigned',
 'Not assigned',
 'North York',
 'North York',
 'Downtown Toronto']

In [12]:
col3[:5]

['', '', 'Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront']

### All the values have a \n in front of the value.

In [13]:
col11=[]
for a in col1:
    spl = a.split('\n')
    col11.append(spl[1])

col11[:5]

['M1A', 'M2A', 'M3A', 'M4A', 'M5A']

In [14]:
col1=col11.copy()

## Create a Data Frame

In [15]:
df=pd.DataFrame(col1)
df.head()

,0
0,M1A
1,M2A
2,M3A
3,M4A
4,M5A


In [16]:
df[1]=col2
df[2]=col3


In [17]:
df.columns=col

In [18]:
df=df[df["Borough"]!='Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
df=df.reset_index()

In [20]:
df.shape

(38, 4)

In [21]:
df.columns

Index(['index', 'Postal Code', 'Borough', 'Neighborhood'], dtype='object')

In [22]:
df=df[col]

In [23]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
df['Neighborhood'].notnull().count()

38

# Part-2
## Adding Location to the DataFrame

In [25]:
location_data = pd.read_csv('Geospatial_Coordinates.csv')
location_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
x=df['Postal Code'].values
x[:5]

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A'], dtype=object)

In [27]:
#Define a empty list.
y=[]

#Loop through the keys present in df.
for a in x:
    q=location_data[location_data['Postal Code'].str.contains(a)]
    y.append(q['Latitude'].values)

y[:5]

[array([43.7532586]),
 array([43.7258823]),
 array([43.6542599]),
 array([43.718518]),
 array([43.6623015])]

In [28]:
df['Latitude']=y
df.head()

,Postal Code,Borough,Neighborhood,Latitude
0,M3A,North York,Parkwoods,[43.7532586]
1,M4A,North York,Victoria Village,[43.725882299999995]
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",[43.6542599]
3,M6A,North York,"Lawrence Manor, Lawrence Heights",[43.718517999999996]
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",[43.6623015]


In [29]:
df['Latitude']=df['Latitude'].astype('float')

In [30]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude
0,M3A,North York,Parkwoods,43.753259
1,M4A,North York,Victoria Village,43.725882
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301


## We repeat the process for Longitude

In [31]:
#Define a empty list.
z=[]

#Loop through the keys present in df.
for a in x:
    q=location_data[location_data['Postal Code'].str.contains(a)]
    z.append(q['Longitude'].values)

df['Longitude']=z
df['Longitude']=df['Longitude'].astype('float')

# Part-3
## Clustering places that consist of the word, Toronto

In [32]:
new_df=df[df['Borough'].str.contains('Toronto')]
new_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [33]:
df.columns

Index(['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

### Correcting the index

In [34]:
new_df=new_df.reset_index()
col=['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
new_df=new_df[col]
new_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Foursquare credentials

In [35]:
CLIENT_ID='Y0GRM0ZOQOOJ5R3HIUT33XV1WBIXGKDJXMFRCLS542TZ5ERA'
CLIENT_SECRET='UWUIATL3BBPTMXOM3KVN3SE02TR21OUGLHN50XDCJTVTU32J'
VERSION='20180605'

In [36]:
radius=500
LIMIT=100

## Function to get explore data from Foursquare

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
toronto_venues = getNearbyVenues(names=new_df['Neighborhood'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity


In [39]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
...,...,...,...,...,...,...,...
647,"Little Portugal, Trinity",43.647927,-79.419750,El Almacen Yerba Mate Cafe + Gallery,43.643682,-79.421627,Café
648,"Little Portugal, Trinity",43.647927,-79.419750,apt 200,43.644026,-79.420063,Bar
649,"Little Portugal, Trinity",43.647927,-79.419750,Hanji,43.644665,-79.416429,Gift Shop
650,"Little Portugal, Trinity",43.647927,-79.419750,Fresh,43.645010,-79.415084,Juice Bar


### Check how many venues are returned per Neighborhood

In [40]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
"Dufferin, Dovercourt Village",19,19,19,19,19,19
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100
"Little Portugal, Trinity",42,42,42,42,42,42
"Queen's Park, Ontario Provincial Government",34,34,34,34,34,34
"Regent Park, Harbourfront",47,47,47,47,47,47


In [41]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 167 uniques categories.


In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017857,0.000000,0.00,0.00,0.017857,0.00,0.000000,0.000000,0.000000
1,Central Bay Street,0.015873,0.000000,0.000000,0.00,0.000000,0.015873,0.000000,0.000000,0.000000,...,0.00,0.031746,0.000000,0.00,0.00,0.015873,0.00,0.000000,0.000000,0.000000
2,Christie,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.058824,...,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
3,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,...,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
4,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.02,0.010000,0.020000,0.00,0.00,0.000000,0.01,0.010000,0.000000,0.000000
5,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.05,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.01,0.000000,0.010000,0.00,0.01,0.010000,0.00,0.000000,0.010000,0.000000
6,"Little Portugal, Trinity",0.023810,0.000000,0.000000,0.00,0.023810,0.000000,0.000000,0.047619,0.000000,...,0.00,0.000000,0.023810,0.00,0.00,0.047619,0.00,0.047619,0.023810,0.000000
7,"Queen's Park, Ontario Provincial Government",0.029412,0.000000,0.000000,0.00,0.000000,0.000000,0.029412,0.000000,0.000000,...,0.00,0.000000,0.029412,0.00,0.00,0.029412,0.00,0.000000,0.000000,0.000000
8,"Regent Park, Harbourfront",0.021277,0.000000,0.021277,0.00,0.021277,0.000000,0.000000,0.021277,0.000000,...,0.00,0.000000,0.042553,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000
9,"Richmond, Adelaide, King",0.000000,0.021505,0.000000,0.00,0.010753,0.010753,0.000000,0.010753,0.000000,...,0.00,0.032258,0.010753,0.00,0.00,0.010753,0.00,0.000000,0.000000,0.010753


In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Beer Bar,Café,Seafood Restaurant,Restaurant,Hotel,Greek Restaurant
1,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Department Store,Salad Place,Burger Joint,Bar,Middle Eastern Restaurant
2,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Diner,Nightclub,Candy Store,Restaurant,Coffee Shop
3,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Park,Brazilian Restaurant,Café,Pool,Pizza Place,Bar,Bank,Supermarket
4,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Restaurant,Hotel


In [46]:
from sklearn.cluster import KMeans

In [47]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 2, 3, 0, 0, 0, 4, 0, 0])

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged =new_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Restaurant,Café,Theater,Breakfast Spot,Dessert Shop,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Creperie,Burrito Place,Beer Bar,Italian Restaurant,Japanese Restaurant,Park,Bar,Bank
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Restaurant,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Clothing Store,Department Store,Beer Bar,Lingerie Store,Cosmetics Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Health Food Store,Trail,Women's Store,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


In [49]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [50]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters